In [ ]:
#restart the kernal at each iteration
from IPython.display import display, Javascript
display(Javascript('''
    var kernel = Jupyter.notebook.kernel;
    var command = "notebook.kernel.restart();";
    kernel.execute(command);
    command = "notebook.execute_all_cells();";
    kernel.execute(command);
'''))


In [ ]:
#record the total computation time
import time
start_time = time.time()

In [ ]:
from all_functions import *

In [ ]:
#record the time to generate input
start_input_generate_time = time.time()

In [ ]:
#generate and read user inputs

if __name__ == "__main__":
    filename = "userInputs.xlsx"  # Specify the filename
    generate_user_input(filename)
    print(f"Excel file '{filename}' generated with 100 random conditions for User A and B.")

    A_data, B_data = read_user_input(filename)

In [ ]:
end_input_generate_time = time.time()
input_generate_time = end_input_generate_time - start_input_generate_time
print("Time taken to generate and read user inputs:", input_generate_time)

In [ ]:
# count how many inputs conditions are true for A and B
A_yes_count, B_yes_count = count_yes_conditions(A_data, B_data)
print("A_yes_count:", A_yes_count)
print("B_yes_count:", B_yes_count)


In [ ]:
#record time for generating ethical profiles
start_profile_generate_time = time.time()

In [ ]:
#generate and read profiles for A and B

if __name__ == "__main__":
    filename = "profiles.xlsx"  # Specify the filename
    generate_profiles(filename)
    print(f"Excel file '{filename}' generated with modified requirements.")

    A_profiles, B_profiles = read_profiles(filename)

    # Sort the profiles by the highest value
    A_profile = dict(sorted(A_profiles.items(), key=lambda x: x[1][1], reverse=True))
    B_profile = dict(sorted(B_profiles.items(), key=lambda x: x[1][1], reverse=True))

In [ ]:
end_profile_generate_time = time.time()
profile_generate_time = end_profile_generate_time - start_profile_generate_time
print("Time taken to generate and read profiles:", profile_generate_time)

In [ ]:
#record time for generating tasks
start_task_generate_time = time.time()

In [ ]:
#generate assign tasks to a specified user in an existing Excel file 

if __name__ == "__main__":
    filename = "tasks.xlsx"  # Specify the filename
    generate_tasks(filename)
    print(f"Excel file '{filename}' generated with tasks and descriptions.")

    A_tasks = assign_tasks(filename, "A_tasks")
    B_tasks = assign_tasks(filename, "B_tasks")

In [ ]:
end_task_generate_time = time.time()
task_generate_time = end_task_generate_time - start_task_generate_time
print("Time taken to generate and read tasks:", task_generate_time)

In [ ]:
#record time for computing task implication
start_Task_implications_time = time.time()

In [ ]:
#compute task implications

A_task_implication = create_task_implication(A_tasks, A_profile)
B_task_implication = create_task_implication(B_tasks, B_profile)

result_df_A = calculate_task_value(A_profile, A_task_implication, 'A')
result_df_B = calculate_task_value(B_profile, B_task_implication, 'B')


In [ ]:
end_Task_implications_time = time.time()
Task_implications_time = end_Task_implications_time - start_Task_implications_time
print("Time taken to calculate task implications:", Task_implications_time)

In [ ]:
#map user input with principles
A_data_to_profiles = assign_data_to_profiles(A_data, A_profiles)
B_data_to_profiles = assign_data_to_profiles(B_data, B_profiles)

data, profiles, Task_implications, Task_value, data_associated_to_principles, Total_tasks = get_all_data(A_data, B_data, A_profiles, B_profiles, A_task_implication, B_task_implication, result_df_A, result_df_B, A_data_to_profiles, B_data_to_profiles, A_tasks, B_tasks)
agents = assign_data_to_agent(data, profiles, Task_implications, Task_value, data_associated_to_principles, Total_tasks)

In [ ]:
#main negotiation class

class Negotiation:

    num_rounds = 10000 # maximum number of runs


    def __init__(self, agents):
        self.agents = agents
        self.played_rounds = 0
        self.agreement_rounds = 0 
        self.A_agreement_rounds = 0
        self.B_agreement_rounds = 0
        self.no_agreement_rounds = 0
        self.A_no_agreement_rounds = 0
        self.B_no_agreement_rounds = 0
        self.A_quit_rounds = 0
        self.B_quit_rounds = 0
        self.old_sender = None
        self.start_decision_time = 0
        self.end_decision_time = 0


    def run(self):

        S_offer = {}
        S_utility = {}
        S_sorted_offers = {}
        S_remaining_offers = {}  
        R_offer = {}
        R_utility = {}
        R_sorted_offers = {}
        R_remaining_offers = {}


        #for 1st round
        for i in range(1, self.num_rounds + 1):
            if i == 1:
                print("Round: ", i)

                #choose sender and receiver
                sender, receiver = select_sender_receiver(self.agents)
                print("Sender agent is: ", sender)
                print("Receiver agent is: ", receiver)
                self.old_sender = sender

                #count time for generating alternative offers for sender
                start_sender_offer_time = time.perf_counter()

                #call offer generation function for sender
                sender_offers = generate_offers(sender, self.agents)

                end_sender_offer_time = time.perf_counter()


                #call the function to send an offer following a tactic for sender
                S_offer, S_utility, S_remaining_offers, S_all_offers = find_best_offer(sender_offers)
                print(sender, "sends offer as : ", S_offer)

                #count time for generating alternative offers for receiver
                start_receiver_offer_time = time.perf_counter()

                #call offer generation function for receiver
                receiver_offers = generate_offers(receiver, self.agents)

                end_receiver_offer_time = time.perf_counter()

                #call the function to send an offer following a tactic for receiver
                R_offer, R_utility, R_remaining_offers, R_all_offers = find_best_offer(receiver_offers)
                print("and", receiver, "would like to do : ", R_offer)
                

                #call calculate utility function 
                print("******** Receiver calculates utility for both offers ********")
                sender_utility, receiver_utility, utility = calculate_new_random_utility(S_offer, receiver, self.agents, R_offer)

                print("Sender's offer utility: ", sender_utility)
                print("Receiver's offer utility that it wants to do: ", receiver_utility)
                print("Calculated Utility: ", utility)
                

                # Decision making
                if utility > 0:
                    print(receiver, "accepts the offer from ", sender)
                    print("Round", i, "ends")
                    print("*********************************")
                    self.agreement_rounds += 1
                    self.played_rounds += 1
                    i += 1

                    if receiver == 'A':
                        self.A_agreement_rounds += 1
                    elif receiver == 'B':
                        self.B_agreement_rounds += 1
                    break

                elif utility <= 0:
                    print(receiver, "rejects the offer of ", sender)
                    print("Round", i, "ends")
                    print("*********************************")
                    self.no_agreement_rounds += 1
                    self.played_rounds += 1
                    i += 1

                    if receiver == 'A':
                        self.A_no_agreement_rounds += 1
                    elif receiver == 'B':
                        self.B_no_agreement_rounds += 1
                

                reuse_offer = R_offer

                print("_______________________________________________________")


            #for 2nd round

            else:
                print("Round: ", i)

                #switch roles
                sender, receiver= switch_role(sender, receiver)
                print("Sender agent is: ", sender)
                print("Receiver agent is: ", receiver)

                #switch data
                S_offer, S_utility, S_sorted_offers, S_remaining_offers, R_offer, R_utility, R_sorted_offers, R_remaining_offers = switch_data(S_offer, S_utility, S_sorted_offers, S_remaining_offers, R_offer, R_utility, R_sorted_offers, R_remaining_offers)
                

                #sender's offer
                S_offer = reuse_offer
                if S_offer == {}:
                    print(sender, "wants to: QUIT Negotiation")
                else:
                    print(sender, "sends offer as : ", S_offer)
                
    
                #receiver offer
                if len(R_remaining_offers) == 0:
                    R_offer = {}
                    print(receiver, "wants to: QUIT Negotiation")

                elif len(R_remaining_offers) == 1:
                    k, val = random.choice(list(R_remaining_offers.items()))
                    R_offer = {k: val['tasks']}
                    print("and", receiver, "would like to do : ", R_offer)
                    del R_remaining_offers[k]
                    #it will return same offer


                elif len(R_remaining_offers) > 1:
                    R_offer, R_utility, R_remaining_offers = updated_dummy_best_offer(R_remaining_offers)
                    print("and", receiver, "would like to do : ", R_offer)


                self.start_decision_time = time.perf_counter()

                
                #call utility function
                if S_offer == {}:
                    print("Round", i, "ends")
                    print("*********************************")
                    self.no_agreement_rounds += 1
                    self.played_rounds += 1
                    i += 1

                    if sender == 'A':
                        self.A_quit_rounds += 1
                    elif sender == 'B':
                        self.B_quit_rounds += 1
                    break

            

                elif R_offer == {}:
                    print("Round", i, "ends")
                    print("*********************************")
                    self.quit_rounds += 1
                    self.no_agreement_rounds += 1
                    self.played_rounds += 1
                    i += 1

                    if receiver == 'A':
                        self.A_quit_rounds += 1
                    elif receiver == 'B':
                        self.B_quit_rounds += 1
                    break

                
                
                elif S_offer != {} and R_offer != {}:
                    print("******** Receiver calculates utility for both offers ********")
                    sender_utility, receiver_utility, utility = calculate_new_random_utility(S_offer, receiver, self.agents, R_offer)

                    print("Sender's offer utility: ", sender_utility)
                    print("Receiver's offer utility that it wants to do: ", receiver_utility)
                    print("Calculated Utility: ", utility)


                    # Decision making
                    if utility > 0:
                        print(receiver, "accepts the offer from ", sender)
                        print("Negotiation ends with agreemnet")
                        print("*********************************")
                        self.agreement_rounds += 1
                        self.played_rounds += 1
                        i += 1

                        if receiver == 'A':
                            self.A_agreement_rounds += 1
                        elif sender == 'B':
                            self.B_agreement_rounds += 1

                        self.end_decision_time = time.perf_counter()
                        break

                    elif utility <= 0:
                        print(receiver, "rejects the offer of ", sender)
                        print("Round", i, "ends")
                        print("*********************************")
                        self.no_agreement_rounds += 1
                        self.played_rounds += 1
                        i += 1

                        if receiver == 'A':
                            self.A_no_agreement_rounds += 1
                        elif receiver == 'B':
                            self.B_no_agreement_rounds += 1

                        self.end_decision_time = time.perf_counter()

                reuse_offer = R_offer
                

                print("_______________________________________________________")

        self.remaining_rounds = self.num_rounds - self.played_rounds
        print("Total argued rounds: ", self.played_rounds)
        print("Agreement rounds: ", self.agreement_rounds)
        print("Rounds accepted by A: ", self.A_agreement_rounds)
        print("Rounds accepted by B: ", self.B_agreement_rounds)
        print("No-agreement reached in rounds: ", self.no_agreement_rounds)
        print("Rounds rejected by A: ", self.A_no_agreement_rounds)
        print("Rounds rejected by B: ", self.B_no_agreement_rounds)
        print("Rounds quit by A: ", self.A_quit_rounds)
        print("Rounds quit by B: ", self.B_quit_rounds)

        self.sender_offer_time = end_sender_offer_time - start_sender_offer_time
        self.receiver_offer_time = end_receiver_offer_time - start_receiver_offer_time
        self.decision_time = self.end_decision_time - self.start_decision_time



obj = Negotiation(agents)
output =obj.run()



In [ ]:
# record the total computation time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Running Time {elapsed_time} seconds.")

In [ ]:
# to generate teh csv file
sender_offer_time = obj.sender_offer_time
receiver_offer_time = obj.receiver_offer_time
decision_time = obj.decision_time
sender_offer_time = f'{sender_offer_time:.7f}'
receiver_offer_time = f'{receiver_offer_time:.7f}'
decision_time = f'{decision_time:.7f}'

print("Input generate time:", input_generate_time)
print("Profile generate time:", profile_generate_time)
print("Task generate time:", task_generate_time)
print("Task implications calculation time:", Task_implications_time)
print("Sender offer generation time:", sender_offer_time)
print("Receiver offer generation time:", receiver_offer_time)
print("Decision-making time:", decision_time)
print("Total running taken:", elapsed_time)

num_rounds = obj.played_rounds
print("Total rounds: ", num_rounds)

input = 50
mapping = 5

print("Input: ", input)
print("Mapping: ", mapping)

sender = obj.old_sender
print("sender: ", sender)

print(f"A_yes_counts: {A_yes_count}")
print(f"B_yes_counts: {B_yes_count}")

In [ ]:
import pandas as pd

results_list = []
num_runs = 1  

results_list.append({
    'Iteration': num_runs,
    'Input Generate Time': input_generate_time,
    'Profile Generate Time': profile_generate_time,
    'Task Generate Time': task_generate_time,
    'Task Implications Time': Task_implications_time,
    'Sender Offer Generation Time': sender_offer_time,
    'Receiver Offer Generation Time': receiver_offer_time,
    'Decision-making Time': decision_time,
    'Running Time': elapsed_time,
    'Total Rounds': num_rounds,
    'Input': input,
    'Mapping': mapping,
    'Sender': sender,
    'A_yes_counts': A_yes_count,
    'B_yes_counts': B_yes_count
})

output_df = pd.DataFrame(results_list)

# Save the DataFrame to a CSV file for one iteration
output_df.to_csv('output.csv', index=False)

